In [4]:
import numpy as np
import pandas as pd
import requests

## Goals

Predict number of riders for a given day based on day of week, holiday, temperature, chance of rain, something like estimated PTO taken by people in that week. 

I'm not using week date as a predictor, that feels like it'd overfit or maybe interfere with the PTO taken part. I'll figure it out.

## WMATA Dataset

Downloaded September 8, 2024, with data up to last week, starting on Jan 1, 2023

In [5]:
wmata_full = pd.read_csv("Daily Ridership - Station-Level Full data.csv")
wmata_full.head()

,Date,Mode,Day of Week,Holiday,Station Name,Weekday / Saturday / Sunday,Entries Or Boardings
0,5/22/2021 12:00:00 AM,Rail,Sat,No,Addison Road,Saturday,15
1,5/29/2021 12:00:00 AM,Rail,Sat,No,Addison Road,Saturday,216
2,6/5/2021 12:00:00 AM,Rail,Sat,No,Addison Road,Saturday,255
3,6/12/2021 12:00:00 AM,Rail,Sat,No,Addison Road,Saturday,334
4,6/19/2021 12:00:00 AM,Rail,Sat,No,Addison Road,Saturday,271


In [6]:
# making the date column simpler, just dates and no times
wmata_full['Date'] = pd.to_datetime(wmata_full['Date']).dt.date
wmata_full['Date'] = pd.to_datetime(wmata_full['Date'])
wmata_full.head()

C:\Users\lcabrera\AppData\Local\Temp\ipykernel_23760\776062870.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  wmata_full['Date'] = pd.to_datetime(wmata_full['Date']).dt.date


,Date,Mode,Day of Week,Holiday,Station Name,Weekday / Saturday / Sunday,Entries Or Boardings
0,2021-05-22,Rail,Sat,No,Addison Road,Saturday,15
1,2021-05-29,Rail,Sat,No,Addison Road,Saturday,216
2,2021-06-05,Rail,Sat,No,Addison Road,Saturday,255
3,2021-06-12,Rail,Sat,No,Addison Road,Saturday,334
4,2021-06-19,Rail,Sat,No,Addison Road,Saturday,271


In [7]:
wmata_full['Month'] = wmata_full['Date'].dt.month

In [38]:
# I need zip codes for each station, so that then I can look at weather in that zip code for the day
# Thankfully, MD has a dataset for all the stations in the DMV
metro_stations_full = pd.read_csv("metro_stations_gisdata.csv")
metro_stations_full.head()

metro_stations = metro_stations_full[["Name","Address","Line"]]

# but they don't freaking have zip codes
metro_stations.head()

,Name,Address,Line
0,Branch Ave,"4704 OLD SOPER ROAD, SUITLAND, MD",green
1,Braddock Road,"700 N. WEST ST., ALEXANDRIA, VA","blue, yellow"
2,King St-Old Town,"1900 KING STREET, ALEXANDRIA, VA","blue, yellow"
3,Eisenhower Ave,"2400 EISENHOWER AVENUE, ALEXANDRIA, VA",yellow
4,Huntington,"2701 HUNTINGTON AVENUE, ALEXANDRIA, VA",yellow


In [39]:
# Replace with your OpenCage API key
API_KEY = '4a8a3ce6dc484e0685fd913d62432caf'
BASE_URL = 'https://api.opencagedata.com/geocode/v1/json'

def get_zip_code(address):
    params = {
        'q': address,
        'key': API_KEY
    }
    response = requests.get(BASE_URL, params=params)
    data = response.json()
    
    if data['results']:
        address = data['results'][0].get('components', {})
        return address.get('postcode')
    return None

# Assuming the addresses are in a column named 'Address'
metro_stations['ZipCode'] = metro_stations['Address'].apply(get_zip_code)

metro_stations.head()

metro_stations.to_csv("metro_stations_with_zips.csv")

C:\Users\lcabrera\AppData\Local\Temp\ipykernel_23760\2730102442.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metro_stations['ZipCode'] = metro_stations['Address'].apply(get_zip_code)


In [91]:
# we got zip codes! Let's see a list of unique ones.
metro_stations["ZipCode"].unique()

array(['20746', '22301', '22314', '22303', '20020', '20003', '20018',
       '20019', '20008', '20015', '20010', '22182', '22102', '20784',
       '20785', '20774', '20002', '22209', '22211', '20036', '22202',
       '20005', '20001', '20745', '20740', '20016', '22043', '20190',
       '20012', '20906', '20900', '20910', '20017', '20317', '20782',
       '20743', '20855', '20850', '20852', '20814', '20037', '22201',
       '22205', '20006', '20202', '20024', '20004', '22304', '22150',
       '20009', '20032', '20748', '22180', '22031', '20007', '20170',
       '20171', '20166', '20147'], dtype=object)

In [76]:
metro_stations[metro_stations['ZipCode'].isna()]
# 10 stations have no zip code. Let's add them manually:

manual_zipcodes_dict = {'Huntington':'22303','McLean':'22102','New Carrollton':'20784','Arlington Cemetery':'22211',
                   'Pentagon':'22202','Southern Avenue':'20745','North Bethesda':'20852',
                   'Ronald Reagan Washington National Airport':'22202','Loudoun Gateway':'20166','Ashburn':'20147'}

In [90]:
for station, zipcode in manual_zipcodes_dict.items():
    metro_stations.loc[metro_stations['Name'] == station, 'ZipCode'] = zipcode

metro_stations[metro_stations['ZipCode'].isna()]
# no na!

,Name,Address,Line,ZipCode


In [94]:
metro_stations[metro_stations['Name'] == "Pentagon"]
# and the zip codes went in

,Name,Address,Line,ZipCode
26,Pentagon,"2 SOUTH ROTARY ROAD, ARLINGTON, VA","blue, yellow",22202


## Joining datasets

In [96]:
wmata_clean = wmata_full.dropna(subset=['Station Name'])
len(wmata_clean['Station Name'].unique())
# 98 stations

gis_stations_list = sorted(metro_stations['Name'].unique())
len(gis_stations_list)
# 98

wmata_stations_list = sorted(wmata_clean['Station Name'].unique())
len(wmata_stations_list)
#98

98

In [49]:
# the names of the stations don't always match
print(wmata_stations_list[0])
gis_stations_list[0]

Addison Road


'Addison Road-Seat Pleasant'

In [80]:
for st in wmata_stations_list:
    if st not in gis_stations_list:
        print("Station " + st + " not in GIS dataset")

# the names just don't match, need a fuzzy logic matching

Station Addison Road not in GIS dataset
Station Archives not in GIS dataset
Station Dulles Airport not in GIS dataset
Station Dunn Loring not in GIS dataset
Station Gallery Place not in GIS dataset
Station Mt Vernon Sq not in GIS dataset
Station Rhode Island Ave not in GIS dataset
Station Southern Ave not in GIS dataset
Station U Street not in GIS dataset
Station Vienna not in GIS dataset
Station Virginia Sq-GMU not in GIS dataset
Station Woodley Park not in GIS dataset


In [81]:
from thefuzz import process

# Function to find the best match for each station in wmata_stations_list from gis_stations_list
def match_stations(station, choices):
    best_match = process.extractOne(station, choices)
    return best_match[0], best_match[1]  # Return the name and similarity score

# Create a DataFrame for the matches
matches = []
for station in wmata_stations_list:
    match_name, score = match_stations(station, gis_stations_list)
    matches.append({'WMATA Station': station, 'GIS Station Match': match_name, 'Match Score': score})

metro_name_matches = pd.DataFrame(matches)

metro_name_matches.to_csv("metro_station_matching_names.csv")
# Show the DataFrame with the matches
display(metro_name_matches)

,WMATA Station,GIS Station Match,Match Score
0,Addison Road,Addison Road-Seat Pleasant,90
1,Anacostia,Anacostia,100
2,Archives,Archives-Navy Memorial-Penn Quarter,90
3,Arlington Cemetery,Arlington Cemetery,100
4,Ashburn,Ashburn,100
...,...,...,...
93,West Falls Church,West Falls Church,100
94,West Hyattsville,West Hyattsville,100
95,Wheaton,Wheaton,100
96,Wiehle-Reston East,Wiehle-Reston East,100


In [82]:
print(gis_stations_list)

['Addison Road-Seat Pleasant', 'Anacostia', 'Archives-Navy Memorial-Penn Quarter', 'Arlington Cemetery', 'Ashburn', 'Ballston-MU', 'Benning Road', 'Bethesda', 'Braddock Road', 'Branch Ave', 'Brookland-CUA', 'Capitol Heights', 'Capitol South', 'Cheverly', 'Clarendon', 'Cleveland Park', 'College Park-U of Md', 'Columbia Heights', 'Congress Heights', 'Court House', 'Crystal City', 'Deanwood', 'Downtown Largo', 'Dunn Loring-Merrifield', 'Dupont Circle', 'East Falls Church', 'Eastern Market', 'Eisenhower Ave', 'Farragut North', 'Farragut West', 'Federal Center SW', 'Federal Triangle', 'Foggy Bottom-GWU', 'Forest Glen', 'Fort Totten', 'Franconia-Springfield', 'Friendship Heights', 'Gallery Pl-Chinatown', 'Georgia Ave-Petworth', 'Glenmont', 'Greenbelt', 'Greensboro', 'Grosvenor-Strathmore', 'Herndon', 'Huntington', 'Hyattsville Crossing', 'Innovation Center', 'Judiciary Square', 'King St-Old Town', "L'Enfant Plaza", 'Landover', 'Loudoun Gateway', 'McLean', 'McPherson Sq', 'Medical Center', 'M

In [83]:
metro_name_matches['final_match'] = ""
# Iterate over the rows using DataFrame's .iterrows()
for i, row in metro_name_matches.iterrows():
    # Check if 'Match Score' is 100
    if row['Match Score'] == 100:
        # Assign the corresponding value from gis_stations_list to 'final_match'
        metro_name_matches.loc[i, 'final_match'] = row['GIS Station Match']

metro_name_matches

,WMATA Station,GIS Station Match,Match Score,final_match
0,Addison Road,Addison Road-Seat Pleasant,90,
1,Anacostia,Anacostia,100,Anacostia
2,Archives,Archives-Navy Memorial-Penn Quarter,90,
3,Arlington Cemetery,Arlington Cemetery,100,Arlington Cemetery
4,Ashburn,Ashburn,100,Ashburn
...,...,...,...,...
93,West Falls Church,West Falls Church,100,West Falls Church
94,West Hyattsville,West Hyattsville,100,West Hyattsville
95,Wheaton,Wheaton,100,Wheaton
96,Wiehle-Reston East,Wiehle-Reston East,100,Wiehle-Reston East


In [84]:
metro_name_matches[metro_name_matches['Match Score']==90]
# all 90s are good matches 

,WMATA Station,GIS Station Match,Match Score,final_match
0,Addison Road,Addison Road-Seat Pleasant,90,
2,Archives,Archives-Navy Memorial-Penn Quarter,90,
24,Dunn Loring,Dunn Loring-Merrifield,90,
59,Mt Vernon Sq,Mt Vernon Sq 7th St-Convention Center,90,
70,Rhode Island Ave,Rhode Island Ave-Brentwood,90,
86,U Street,U Street/African-Amer Civil War Memorial/Cardozo,90,
90,Vienna,Vienna/Fairfax-GMU,90,
97,Woodley Park,Woodley Park-Zoo/Adams Morgan,90,


In [85]:
# then run it again for the 90s
for i, row in metro_name_matches.iterrows():
    # Check if 'Match Score' is 90
    if row['Match Score'] == 90 and row['final_match'] == "":
        # Assign the corresponding value from gis_stations_list to 'final_match'
        metro_name_matches.loc[i, 'final_match'] = row['GIS Station Match']

metro_name_matches[metro_name_matches['Match Score']==90]


,WMATA Station,GIS Station Match,Match Score,final_match
0,Addison Road,Addison Road-Seat Pleasant,90,Addison Road-Seat Pleasant
2,Archives,Archives-Navy Memorial-Penn Quarter,90,Archives-Navy Memorial-Penn Quarter
24,Dunn Loring,Dunn Loring-Merrifield,90,Dunn Loring-Merrifield
59,Mt Vernon Sq,Mt Vernon Sq 7th St-Convention Center,90,Mt Vernon Sq 7th St-Convention Center
70,Rhode Island Ave,Rhode Island Ave-Brentwood,90,Rhode Island Ave-Brentwood
86,U Street,U Street/African-Amer Civil War Memorial/Cardozo,90,U Street/African-Amer Civil War Memorial/Cardozo
90,Vienna,Vienna/Fairfax-GMU,90,Vienna/Fairfax-GMU
97,Woodley Park,Woodley Park-Zoo/Adams Morgan,90,Woodley Park-Zoo/Adams Morgan


In [86]:
metro_name_matches[metro_name_matches['Match Score']<90]
# all of these are good except Dulles, which should be "Washington Dulles International Airport"
metro_name_matches.loc[metro_name_matches['WMATA Station']=="Dulles Airport", 'final_match'] = "Washington Dulles International Airport"

metro_name_matches[metro_name_matches['Match Score']<90]

,WMATA Station,GIS Station Match,Match Score,final_match
23,Dulles Airport,Ronald Reagan Washington National Airport,86,Washington Dulles International Airport
38,Gallery Place,Gallery Pl-Chinatown,86,
78,Southern Ave,Southern Avenue,89,
91,Virginia Sq-GMU,Virginia Square-GMU,88,


In [87]:
# then run it again for the remaining rows
for i, row in metro_name_matches.iterrows():
    # Check if 'Match Score' is 90
    if row['Match Score'] < 90 and row['final_match'] == "":
        # Assign the corresponding value from gis_stations_list to 'final_match'
        metro_name_matches.loc[i, 'final_match'] = row['GIS Station Match']

# now that they all have a match, save the file:
metro_name_matches.to_csv("metro_station_matching_names.csv")

In [97]:
metro_name_matches.head()

,WMATA Station,GIS Station Match,Match Score,final_match
0,Addison Road,Addison Road-Seat Pleasant,90,Addison Road-Seat Pleasant
1,Anacostia,Anacostia,100,Anacostia
2,Archives,Archives-Navy Memorial-Penn Quarter,90,Archives-Navy Memorial-Penn Quarter
3,Arlington Cemetery,Arlington Cemetery,100,Arlington Cemetery
4,Ashburn,Ashburn,100,Ashburn


In [98]:
# now let's get the zip codes into the dataset we'll be using, wmata_clean
metro_stations_merged = pd.merge(metro_stations, metro_name_matches, left_on='Name', right_on='final_match', how='left')
metro_stations_merged[metro_stations_merged['ZipCode'].isna()]
# no stations without a zip code!

,Name,Address,Line,ZipCode,WMATA Station,GIS Station Match,Match Score,final_match


In [100]:
metro_stations_merged.head()

,Name,Address,Line,ZipCode,WMATA Station,GIS Station Match,Match Score,final_match
0,Branch Ave,"4704 OLD SOPER ROAD, SUITLAND, MD",green,20746,Branch Ave,Branch Ave,100,Branch Ave
1,Braddock Road,"700 N. WEST ST., ALEXANDRIA, VA","blue, yellow",22301,Braddock Road,Braddock Road,100,Braddock Road
2,King St-Old Town,"1900 KING STREET, ALEXANDRIA, VA","blue, yellow",22301,King St-Old Town,King St-Old Town,100,King St-Old Town
3,Eisenhower Ave,"2400 EISENHOWER AVENUE, ALEXANDRIA, VA",yellow,22314,Eisenhower Ave,Eisenhower Ave,100,Eisenhower Ave
4,Huntington,"2701 HUNTINGTON AVENUE, ALEXANDRIA, VA",yellow,22303,Huntington,Huntington,100,Huntington


In [101]:
wmata_clean.head()

,Date,Mode,Day of Week,Holiday,Station Name,Weekday / Saturday / Sunday,Entries Or Boardings,Month
0,2021-05-22,Rail,Sat,No,Addison Road,Saturday,15,5
1,2021-05-29,Rail,Sat,No,Addison Road,Saturday,216,5
2,2021-06-05,Rail,Sat,No,Addison Road,Saturday,255,6
3,2021-06-12,Rail,Sat,No,Addison Road,Saturday,334,6
4,2021-06-19,Rail,Sat,No,Addison Road,Saturday,271,6


In [103]:
wmata_merged = pd.merge(wmata_clean, metro_stations_merged[['WMATA Station',"Line","ZipCode"]],
                         left_on='Station Name', right_on='WMATA Station', how='left')

wmata_merged.head()

,Date,Mode,Day of Week,Holiday,Station Name,Weekday / Saturday / Sunday,Entries Or Boardings,Month,WMATA Station,Line,ZipCode
0,2021-05-22,Rail,Sat,No,Addison Road,Saturday,15,5,Addison Road,"blue, silver",20743
1,2021-05-29,Rail,Sat,No,Addison Road,Saturday,216,5,Addison Road,"blue, silver",20743
2,2021-06-05,Rail,Sat,No,Addison Road,Saturday,255,6,Addison Road,"blue, silver",20743
3,2021-06-12,Rail,Sat,No,Addison Road,Saturday,334,6,Addison Road,"blue, silver",20743
4,2021-06-19,Rail,Sat,No,Addison Road,Saturday,271,6,Addison Road,"blue, silver",20743


## Getting a weather dataset